In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#USE IT JUST FOR MAKE LABELS TO ONEHOT VECTOR
from tensorflow.keras.utils import to_categorical
import random
# PLT_DATA=np.array()
from sklearn.datasets import load_iris

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


def cross_entropy(y_true, y_pred):
    epsilon = 1e-12
    num_examples = y_true.shape[0]
    loss = -np.sum(y_true * np.log(y_pred + epsilon)) / num_examples
    return loss


def sigmoid_derivative(x):

    return x * (1 - x)


def accuracy(y_true, y_pred):
    acc = np.mean(y_true == y_pred)
    return acc


def minibatch_GD_train(X_train, y_train, learning_rate, batch_size, num_epochs):

    np.random.seed(0)
    weights1 = np.random.randn(4, 10)
    weights2 = np.random.randn(10, 10)
    weights3 = np.random.randn(10, 3)


    for epoch in range(num_epochs):

        permutation = np.random.permutation(X_train.shape[0])
        X_train = X_train[permutation]
        y_train = y_train[permutation]

        for i in range(0, X_train.shape[0], batch_size):
            # get batch
            X_batch = X_train[i:i + batch_size]
            y_batch = y_train[i:i + batch_size]

            z1 = np.dot(X_batch, weights1)
            a1 = sigmoid(z1)
            z2 = np.dot(a1, weights2)
            a2 = sigmoid(z2)
            z3 = np.dot(a2, weights3)
            y_pred = softmax(z3)
            y_batch = to_categorical(y_batch)
            loss = cross_entropy(y_batch, y_pred)
            acc = accuracy(y_batch, y_pred)




            dL_dy_pred = y_pred - y_batch
            dL_dz3 = dL_dy_pred
            dL_da2 = np.dot(dL_dz3, weights3.T)
            dL_dz2 = dL_da2 * sigmoid_derivative(a2)
            dL_da1 = np.dot(dL_dz2, weights2.T)
            dL_dz1 = dL_da1 * sigmoid_derivative(a1)


            weights3 -= learning_rate * np.dot(a2.T, dL_dz3)
            weights2 -= learning_rate * np.dot(a1.T, dL_dz2)
            weights1 -= learning_rate * np.dot(X_batch.T, dL_dz1)

    return weights1, weights2, weights3



def SGD_train(X_train, y_train, learning_rate, num_epochs):
    # initialize weights
    np.random.seed(0)
    weights1 = np.random.randn(4, 10)
    weights2 = np.random.randn(10, 10)
    weights3 = np.random.randn(10, 3)

    for epoch in range(num_epochs):
        permutation = np.random.permutation(X_train.shape[0])
        X_train = X_train[permutation]
        y_train = y_train[permutation]

        for i in range(0, X_train.shape[0]):

            X_example = X_train[i:i+1]
            y_example = y_train[i:i+1]
            z1 = np.dot(X_example, weights1)
            a1 = sigmoid(z1)
            z2 = np.dot(a1, weights2)
            a2 = sigmoid(z2)
            z3 = np.dot(a2, weights3)
            y_pred = softmax(z3)
            y_example = to_categorical(y_example,num_classes=3)
            loss = cross_entropy(y_example, y_pred)
            acc = accuracy(y_example, y_pred)




            dL_dy_pred = y_pred - y_example
            dL_dz3 = dL_dy_pred
            dL_da2 = np.dot(dL_dz3, weights3.T)
            dL_dz2 = dL_da2 * sigmoid_derivative(a2)
            dL_da1 = np.dot(dL_dz2, weights2.T)
            dL_dz1 = dL_da1 * sigmoid_derivative(a1)


            weights3 -= learning_rate * np.dot(a2.T, dL_dz3)
            weights2 -= learning_rate * np.dot(a1.T, dL_dz2)
            weights1 -= learning_rate * np.dot(X_example.T, dL_dz1)


    return weights1, weights2, weights3



def GD_train(X_train, y_train, test_labels, test_data, learning_rate, num_epochs, faultes=None):
    # np.random.seed(165495)
    # np.random.seed(20000)
    np.random.seed(152)
    weights1 = np.random.randn(4, 10)
    weights2 = np.random.randn(10, 10)
    weights3 = np.random.randn(10, 3)
    y_train = to_categorical(y_train, num_classes=3)
    accuracies = []
    faultes=[]
    for epoch in range(num_epochs):

            z1 = np.dot(X_train, weights1)
            a1 = sigmoid(z1)
            z2 = np.dot(a1, weights2)
            a2 = sigmoid(z2)
            z3 = np.dot(a2, weights3)
            y_pred = softmax(z3)
            loss = cross_entropy(y_train, y_pred)
            acc = accuracy(y_train, y_pred)


            dL_dy_pred = y_pred - y_train
            dL_dz3 = dL_dy_pred
            dL_da2 = np.dot(dL_dz3, weights3.T)
            dL_dz2 = dL_da2 * sigmoid_derivative(a2)
            dL_da1 = np.dot(dL_dz2, weights2.T)
            dL_dz1 = dL_da1 * sigmoid_derivative(a1)


            weights3 -= learning_rate * np.dot(a2.T, dL_dz3)
            weights2 -= learning_rate * np.dot(a1.T, dL_dz2)
            weights1 -= learning_rate * np.dot(X_train.T, dL_dz1)

            y_test_onehot = to_categorical(test_labels)
            z1 = np.dot(test_data, weights1)
            a1 = sigmoid(z1)
            z2 = np.dot(a1, weights2)
            a2 = sigmoid(z2)
            z3 = np.dot(a2, weights3)
            y_pred = softmax(z3)
            y_pred_one_hot = np.argmax(y_pred, axis=1)
            y_pred_one_hot = to_categorical(y_pred_one_hot)
            test_acc = accuracy(y_test_onehot, y_pred_one_hot)
            test_loss = cross_entropy(y_test_onehot, y_pred_one_hot)
            accuracies.append(test_acc)
            faultes.append(1-test_acc)

    plt.plot(accuracies)
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.show()
    plt.plot(faultes)
    plt.title('faultes over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('fualtes')
    plt.show()

    return weights1, weights2, weights3


#################################################
#Q2_A:reading and normalizing and division t0 80% 20%
##################################################

iris = load_iris()
data = iris.data
labels = iris.target
random.seed(42)
data_indices = list(range(len(data)))
random.shuffle(data_indices)
split_point = int(0.8 * len(data))
train_indices = data_indices[:split_point]
test_indices = data_indices[split_point:]
X = data[train_indices]#train_set
y = labels[train_indices]
test_data = data[test_indices]
test_labels = labels[test_indices]
#normalization
test_data = (test_data - np.mean(test_data, axis=0)) / np.std(test_data, axis=0)
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
X = X.astype(np.float64)
y = y.astype(np.float64)
test_data = test_data.astype(np.float64)
test_labels = test_labels.astype(np.float64)



###using SGD_function########
#############################
weights1, weights2, weights3 = SGD_train(X, y, learning_rate=0.01, num_epochs=50)

y_test_onehot = to_categorical(test_labels)
z1 = np.dot(test_data, weights1)
a1 = sigmoid(z1)
z2 = np.dot(a1, weights2)
a2 = sigmoid(z2)
z3 = np.dot(a2, weights3)
y_pred = softmax(z3)
y_pred_one_hot = np.argmax(y_pred, axis=1)
y_pred_one_hot=to_categorical(y_pred_one_hot)
test_acc = accuracy(y_test_onehot, y_pred_one_hot)
test_loss = cross_entropy(y_test_onehot, y_pred_one_hot)
print("\nSGD_train:")
print(f'Test accuracy: {test_acc:.2f}')
print(f'Test loss: {test_loss:.4f}')


###using minibatch_function########
#############################
weights1, weights2, weights3 = minibatch_GD_train(X, y,learning_rate=0.01, batch_size=32, num_epochs=50)

y_test_onehot = to_categorical(test_labels)
z1 = np.dot(test_data, weights1)
a1 = sigmoid(z1)
z2 = np.dot(a1, weights2)
a2 = sigmoid(z2)
z3 = np.dot(a2, weights3)
y_pred = softmax(z3)
y_pred_one_hot = np.argmax(y_pred, axis=1)
y_pred_one_hot=to_categorical(y_pred_one_hot)
test_acc = accuracy(y_test_onehot, y_pred_one_hot)
test_loss = cross_entropy(y_test_onehot, y_pred_one_hot)
print("\nMinibatch_GD_train:")
print(f'Test accuracy: {test_acc:.2f}')
print(f'Test loss: {test_loss:.4f}')


###using GD_function########
#############################




weights1, weights2, weights3 = GD_train(X, y,test_labels,test_data, learning_rate=0.01, num_epochs=100)


y_test_onehot = to_categorical(test_labels)
z1 = np.dot(test_data, weights1)
a1 = sigmoid(z1)
z2 = np.dot(a1, weights2)
a2 = sigmoid(z2)
z3 = np.dot(a2, weights3)
y_pred = softmax(z3)
y_pred_one_hot = np.argmax(y_pred, axis=1)
y_pred_one_hot=to_categorical(y_pred_one_hot)
test_acc = accuracy(y_test_onehot, y_pred_one_hot)
test_loss = cross_entropy(y_test_onehot, y_pred_one_hot)
print("\nGD_train:")
print(f'Test accuracy: {test_acc:.2f}')
print(f'Test loss: {test_loss:.4f}')



